In [ ]:

sp_dvf_file=spark.read.csv('abfs://containerintro@introductiondatastorage.dfs.core.windows.net/immodata/valeursfoncieres-2021.txt',sep="|", header=True)

# COMMAND ----------

display(sp_dvf_file)

# COMMAND ----------


renamed_sp_dvf = sp_dvf_file.select([F.col(col).alias(col.replace(' ', '_')) for col in sp_dvf_file.columns])
dot_sp_dvf = renamed_sp_dvf.withColumn('Valeur_fonciere', regexp_replace('Valeur_fonciere', ',', '.').cast("int"))
dot_sp_dvf = dot_sp_dvf.withColumn('Surface_reelle_bati', dot_sp_dvf['Surface_reelle_bati'].cast("int"))

# COMMAND ----------

dot_sp_dvf.write.format("delta").save("/FileStore/tables/dvf_2021")

# COMMAND ----------

# MAGIC %sql
# MAGIC CREATE TABLE IF NOT EXISTS bronze.dvf USING DELTA LOCATION '/FileStore/tables/dvf_2021'


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.functions import regexp_replace
from pyspark.sql.types import FloatType

In [ ]:
spark.conf.set("fs.azure.account.auth.type.introductiondatastorage.dfs.core.windows.net", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.introductiondatastorage.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.introductiondatastorage.dfs.core.windows.net")

In [ ]:
sp_dvf_file=spark.read.csv('abfs://containerintro@introductiondatastorage.dfs.core.windows.net/immodata/valeursfoncieres-2021.txt',sep="|", header=True)


In [ ]:
!pip install azure-storage-file-datalake

In [ ]:
import pandas as pd

In [ ]:
df24 = pd.read_csv(r'C:\Users\steve\Desktop\valeursfoncieres-2024-s1\valeursfoncieres-2024-s1.txt', sep="|")
df23 = pd.read_csv(r'C:\Users\steve\Desktop\valeursfoncieres-2024-s1\valeursfoncieres-2023.txt', sep="|")
df22 = pd.read_csv(r'C:\Users\steve\Desktop\valeursfoncieres-2024-s1\valeursfoncieres-2022.txt', sep="|")

In [ ]:
concat = pd.concat([df24, df23, df22])

In [ ]:
concat.columns

In [ ]:
for each in ['Surface Carrez du 1er lot',
       '2eme lot', 'Surface Carrez du 2eme lot', '3eme lot',
       'Surface Carrez du 3eme lot', '4eme lot', 'Surface Carrez du 4eme lot',
       '5eme lot', 'Surface Carrez du 5eme lot', 'Nombre de lots',
       'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain']:
    concat[each] = concat[each].astype(str)

In [ ]:
concat.to_parquet('valeursfoncieres_2022_2024.parquet', engine='fastparquet')

In [ ]:
import pandas as pd
from azure.storage.filedatalake import DataLakeServiceClient
from io import StringIO
from io import BytesIO

# Define your ADLS Gen2 credentials and file details
account_name = 'adlsintrosteven'
account_key = ''
file_system_name = 'adlsintro'
file_path = 'valeursfoncieres_2022_2024.parquet'

# Create a DataLakeServiceClient
service_client = DataLakeServiceClient(account_url=f"https://{account_name}.dfs.core.windows.net", credential=account_key)

# Get a file system client
file_system_client = service_client.get_file_system_client(file_system=file_system_name)

# Get a file client
file_client = file_system_client.get_file_client(file_path)

# Read the file content
download = file_client.download_file()
downloaded_bytes = download.readall()

# Convert bytes to a StringIO object
file_content = BytesIO(downloaded_bytes)

# Read the file content into a pandas DataFrame
df = pd.read_parquet(file_content)

# Display the DataFrame
print(df)

In [ ]:
from pyarrow.parquet import ParquetFile
import pyarrow as pa 
pf = ParquetFile(file_content) 
first_ten_rows = next(pf.iter_batches(batch_size = 10)) 
df = pa.Table.from_batches([first_ten_rows]).to_pandas() 